Salva ego network e active ego network degli utenti regolari e attivi.

La ego network e active ego network viene suddivisa in pre lockdown e post lockdown

In [ ]:
import sys


PATH = "../../.."
sys.path.append(f'{PATH}/')

from src.utils.pyspark_init import set_pySpark
import pyspark.pandas as ps
from src.dataProcessing.ego_network.ActiveEgoNetwork import ActiveEgoNetwork
from src.dataProcessing.ego_network.BasicEgoNetwork import BasicEgoNetwork
from src.utils.globalVariable import REGULAR_ACTIVE_USER_DIR, TWEET_DATABASE_DIR
from src.utils.slidingWindow import get_period_list, utc_to_string



set_pySpark(memory="30g")

In [ ]:
def save_egoNetwork(PERIOD_DIR, startPeriod_utc, endPeriod_utc):
    user_id = ps.read_parquet(f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/user")["id"]
    egoNetwork = BasicEgoNetwork(f"{PATH}/{TWEET_DATABASE_DIR}", user_id.to_list())

    egoNetwork = egoNetwork.get_observedEgoNetwork(startPeriod_utc, endPeriod_utc, min_relationship_duration=0.5)
    egoNetwork.to_parquet(f"{PERIOD_DIR}/egoNetwork")


def save_activeEgoNetwork(REGULAR_ACTIVE_USER_DIR):
    activeEgoNetwork = ActiveEgoNetwork(f"{REGULAR_ACTIVE_USER_DIR}/egoNetwork")
    activeEgoNetwork.get_ActiveEgoNetwork().to_parquet(f"{REGULAR_ACTIVE_USER_DIR}/activeEgoNetwork")

Ego netowrk e Active ego network con sliding window + utenti regolari e attivi nel nel periodo post lockdown

In [ ]:
period_list = get_period_list()
for first_period, second_period in period_list:
    first_period_str = utc_to_string(first_period)
    second_period_str = utc_to_string(second_period)

    PERIOD_DIR = f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/{first_period_str}-{second_period_str}"

    save_egoNetwork(PERIOD_DIR, first_period, second_period)
    save_activeEgoNetwork(PERIOD_DIR)